# Train a gesture recognition model for microcontroller use

This notebook demonstrates how to train a 20kb gesture recognition model for [TensorFlow Lite for Microcontrollers](https://tensorflow.org/lite/microcontrollers/overview). It will produce the same model used in the [magic_wand](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/experimental/micro/examples/magic_wand) example application.

The model is designed to be used with [Google Colaboratory](https://colab.research.google.com).

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/experimental/micro/examples/magic_wand/train/train_magic_wand_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/experimental/micro/examples/magic_wand/train/train_magic_wand_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


Training is much faster using GPU acceleration. Before you proceed, ensure you are using a GPU runtime by going to **Runtime -> Change runtime type** and selecting **GPU**. Training will take around 5 minutes on a GPU runtime.

## Install dependencies

Run the following cell to ensure the required dependencies are installed.

In [0]:
!pip uninstall -y tensorflow
!pip install -q tensorflow-gpu==2.0.0-beta1

We'll also clone the TensorFlow repository, which contains the training scripts, and copy them into our workspace.

In [0]:
# Clone the repository from GitHub
!git clone --depth 1 -q https://github.com/tensorflow/tensorflow
# Copy the training scripts into our workspace
!cp -r tensorflow/tensorflow/lite/experimental/micro/magic_wand/train train

## Prepare the data

Next, we'll download the data and extract it into the expected location within the training scripts' directory.

In [0]:
# Download the data we will use to train the model
!wget http://download.tensorflow.org/models/tflite/magic_wand/data.tar.gz
# Extract the data into the train directory
!tar xvzf data.tar.gz -C train 1>/dev/null

We'll then run the scripts that split the data into training, validation, and test sets.

In [0]:
# The scripts must be run from within the train directory
%cd train
# Prepare the data
!python data_prepare.py
# Split the data by person
!python data_split_person.py

## Load TensorBoard

Now, we set up TensorBoard so that we can graph our accuracy and loss as training proceeds.

In [0]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs/scalars

## Begin training

The following cell will begin the training process. Training will take around 5 minutes on a GPU runtime. You'll see the metrics in TensorBoard after a few epochs.

In [0]:
!python train.py --model CNN --person true

## Create a C source file

The `train.py` script writes a quantized model, `model_quantized.tflite`, to the training scripts' directory.

In the following cell, we convert this model into a C++ source file we can use with TensorFlow Lite for Microcontrollers.

In [0]:
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i model_quantized.tflite > /content/model_quantized.cc
# Print the source file
!cat /content/model_quantized.cc